In [1]:
from algosdk import account, mnemonic

private_key, address = account.generate_account()
print("My address: {}".format(address))
print("My private key: {}".format(private_key))
print("My passphrase: {}".format(mnemonic.from_private_key(private_key)))


My address: 2WKJ7VNPAWA7M3BC2BTKLBLYCI2LOFS5OIQEYV3PGQVGJIPKOOAWFDGRLM
My private key: bg6ugBWI3NPCf0WKGru6w25FfeR5WsDmh/GPzaa6hUvVlJ/VrwWB9mwi0GalhXgSNLcWXXIgTFdvNCpkoepzgQ==
My passphrase: soccer return scare life ribbon reject wrap earth eyebrow unaware tail umbrella member butter song foil submit buzz web suggest eternal huge found able member


In [3]:
print(f"Private key from mnemonic: {mnemonic.to_private_key} 'soccer return scare life ribbon reject wrap earth eyebrow unaware tail umbrella member butter song foil submit buzz web suggest eternal huge found able member'")

print(f"Public key from mnemonic: {mnemonic.to_public_key} 'soccer return scare life ribbon reject wrap earth eyebrow unaware tail umbrella member butter song foil submit buzz web suggest eternal huge found able member'")

Private key from mnemonic: <function to_private_key at 0x7fc1b29fc0d0> 'soccer return scare life ribbon reject wrap earth eyebrow unaware tail umbrella member butter song foil submit buzz web suggest eternal huge found able member'
Private key from mnemonic: <function to_public_key at 0x7fc1b29fc160> 'soccer return scare life ribbon reject wrap earth eyebrow unaware tail umbrella member butter song foil submit buzz web suggest eternal huge found able member'


In [4]:
mnemo_2 = "giraffe afford safe mule squeeze stem uncle month across image above orphan tube cook clutch steak cradle merge mad unknown cricket decrease nominee abandon inspire"
priv_2 = mnemonic.to_private_key(mnemo_2)
publ_2 = mnemonic.to_public_key(mnemo_2)

In [6]:
from algosdk.v2client import algod

algod_address = "http://localhost:4001"
algod_token = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"
algod_client = algod.AlgodClient(algod_token, algod_address)


In [8]:
account_info = algod_client.account_info(publ_2)
print("Account balance: {} microAlgos".format(account_info.get('amount')) + "\n")


Account balance: 4000052000000000 microAlgos



In [10]:
account_info = algod_client.account_info(address)
print("Account balance: {} microAlgos".format(account_info.get('amount')) + "\n")

Account balance: 0 microAlgos



In [11]:
# build transaction
from algosdk.future import transaction
from algosdk import constants


params = algod_client.suggested_params()
    # comment out the next two (2) lines to use suggested fees
params.flat_fee = True
params.fee = constants.MIN_TXN_FEE 
receiver = address
note = "Hello World".encode()
amount = 1000000
unsigned_txn = transaction.PaymentTxn(publ_2, params, receiver, amount, None, note)


In [12]:
# sign transaction
signed_txn = unsigned_txn.sign(priv_2)


In [13]:
import json
import base64


    #submit transaction
txid = algod_client.send_transaction(signed_txn)
print("Successfully sent transaction with txID: {}".format(txid))

    # wait for confirmation 
try:
    confirmed_txn = transaction.wait_for_confirmation(algod_client, txid, 4)  
except Exception as err:
    print(err)
    # return

print("Transaction information: {}".format(
    json.dumps(confirmed_txn, indent=4)))
print("Decoded note: {}".format(base64.b64decode(
    confirmed_txn["txn"]["txn"]["note"]).decode()))
print("Starting Account balance: {} microAlgos".format(account_info.get('amount')) )
print("Amount transfered: {} microAlgos".format(amount) )    
print("Fee: {} microAlgos".format(params.fee) ) 


account_info = algod_client.account_info(publ_2)
print("Final Account balance: {} microAlgos".format(account_info.get('amount')) + "\n")


Successfully sent transaction with txID: VYCKG3QII3EJGBLKBSW6NBNNH6HEPPQH7XLPWHSO3PFZJOLU3D7A
Transaction information: {
    "confirmed-round": 668,
    "pool-error": "",
    "sender-rewards": 64000000000,
    "txn": {
        "sig": "n6BEBc0uXemEdHg08Mbdjyeq/AreRV3iDkO3FSv6x1xGjC6G43SjWiTwBg1HZRHyZDexy3tx4O55DmjJKbv1Bg==",
        "txn": {
            "amt": 1000000,
            "fee": 1000,
            "fv": 595,
            "gen": "sandnet-v1",
            "gh": "ZCP0vT7l0mH4Q4rMSm7H0N17ehrAjmESfJ9qnwzCAiE=",
            "lv": 1595,
            "note": "SGVsbG8gV29ybGQ=",
            "rcv": "2WKJ7VNPAWA7M3BC2BTKLBLYCI2LOFS5OIQEYV3PGQVGJIPKOOAWFDGRLM",
            "snd": "I4RGNIWSDHZ527XHXPMPGO5BSA3MEZPXTFDYQZRUWBPFXRURQCVRXA2XUY",
            "type": "pay"
        }
    }
}
Decoded note: Hello World
Starting Account balance: 0 microAlgos
Amount transfered: 1000000 microAlgos
Fee: 1000 microAlgos
Final Account balance: 4000063998999000 microAlgos



In [15]:
account_info = algod_client.account_info(address)
print("Account balance: {} microAlgos".format(account_info.get('amount')) + "\n")


Account balance: 1000001 microAlgos



In [17]:
from algosdk.future.transaction import AssetConfigTxn,AssetFreezeTxn,AssetTransferTxn,wait_for_confirmation

In [18]:
# CREATE ASSET
# Get network params for transactions before every transaction.
params = algod_client.suggested_params()
# comment these two lines if you want to use suggested params
# params.fee = 1000
# params.flat_fee = True
# Account 1 creates an asset called latinum and
# sets Account 2 as the manager, reserve, freeze, and clawback address.
# Asset Creation transaction
txn = AssetConfigTxn(
    sender=publ_2,
    sp=params,
    total=1,
    default_frozen=False,
    unit_name="LATINUM",
    asset_name="latinum",
    manager=publ_2,
    reserve=publ_2,
    freeze= publ_2,
    clawback= publ_2,
    url="https://path/to/my/asset/details", 
    decimals=0)
# Sign with secret key of creator
stxn = txn.sign(publ_2)
# Send the transaction to the network and retrieve the txid.
try:
    txid = algod_client.send_transaction(stxn)
    print("Signed transaction with txID: {}".format(txid))
    # Wait for the transaction to be confirmed
    confirmed_txn = wait_for_confirmation(algod_client, txid, 4)  
    print("TXID: ", txid)
    print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))   
except Exception as err:
    print(err)
# Retrieve the asset ID of the newly created asset by first
# ensuring that the creation transaction was confirmed,
# then grabbing the asset id from the transaction.
print("Transaction information: {}".format(
    json.dumps(confirmed_txn, indent=4)))
# print("Decoded note: {}".format(base64.b64decode(
#     confirmed_txn["txn"]["txn"]["note"]).decode()))


Error: Incorrect padding